#### Sigmoid in Generator, Soft Labels, Gaussian Noise

# TODO
#### add and train a classifier use k fold also, generator k lie combined data and disc k lie train data.. coz testing me sara deke bias ho jaega

In [1]:
%config Completer.use_jedi = False

import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
import keras
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Reshape, multiply, Embedding, merge, Concatenate, Conv1D, BatchNormalization
from keras.layers import Dense, Flatten, Multiply, LSTM
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from keras.layers import UpSampling1D
from keras.layers.core import Activation
from keras.layers import Add
import tensorflow as tf
from evaluation_metrics import *
metric_to_calculate = ['FID', 'MMD', 'DTW', 'PC', 'RMSE', 'TWED']

In [2]:
def discriminator(data_dim, beat_dim=186):
    
    in_label = Input(shape=(beat_dim,1))
    D_in = Input(shape=[data_dim,1])
    x = Concatenate()([D_in, in_label])
    
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*16, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Flatten()(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[D_in, in_label], outputs=out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
    return model

# d_model = discriminator(data_dim=186, beat_dim=186)
# plot_model(d_model, to_file='Final/disc.pdf', show_shapes=True)

In [3]:
def generator(noise_dim=186, beat_dim=186, out_dim=186):
    
    in_label = Input(shape=(beat_dim,1))
    G_in = Input(shape=[noise_dim,1])
    x = Concatenate()([G_in, in_label])
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*16, kernel_size=2, strides=2, padding='valid', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=1, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = LSTM(1, return_sequences=True)(x)
#     x = LSTM(out_dim, return_sequences=False)(x)
#     x = Reshape((out_dim,1))(x)
    out = Activation('tanh')(x)  # for tanh change here
    model = Model(inputs=[G_in, in_label], outputs=out)

    return model

# g_model = generator(noise_dim=186, beat_dim=186, out_dim=186)
# plot_model(g_model, to_file='Temp/gen.pdf', show_shapes=True)

In [4]:
def create_gan(d_model, g_model):
    
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_beat = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_beat])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_beat], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt)
    return model

# gan_model = create_gan(d_model, g_model)
# plot_model(gan_model, to_file='Final/gan.pdf', show_shapes=True)

In [5]:
def reshape(X):
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
        return X
    else:
        if X.shape[-1] == 1:
            return X
        else:
            X = X.reshape(X.shape[0], X.shape[1], 1)
            return X

In [6]:
def load_real_samples():
    X = np.load('Data/X.npy')
    y = np.load('Data/y.npy')

    # print (X.shape, y.shape)

    X_N = X[y==0]
    X_S = X[y==1]
    X_V = X[y==2]

    y_N = y[y==0]
    y_S = y[y==1]
    y_V = y[y==2]

    # print (X_N.shape, y_N.shape)
    # print (X_S.shape, y_S.shape)
    # print (X_V.shape, y_V.shape)

#     X_N=X_N.reshape(X_N.shape[0],X_N.shape[1],1)
#     X_S=X_S.reshape(X_S.shape[0],X_S.shape[1],1)
#     X_V=X_V.reshape(X_V.shape[0],X_V.shape[1],1)

    # print (X_N.shape, y_N.shape)
    # print (X_S.shape, y_S.shape)
    # print (X_V.shape, y_V.shape)
    return reshape(X_N), y_N, reshape(X_S), y_S, reshape(X_V), y_V

In [7]:
def generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_samples):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], int(n_samples/3))
    i_S = randint(0, y_S.shape[0], int(n_samples/3))
    i_V = randint(0, y_V.shape[0], int(n_samples/3))
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    labels = keras.utils.to_categorical(np.hstack((y_N[i_N], y_S[i_S], y_V[i_V])))
    # print (labels.shape)
    
    # generate class labels
    y = reshape(np.random.uniform(0.8, 1, n_samples))
#     y = y.reshape(y.shape[0], 1)
#     y = np.ones((n_samples, 1))
    return [X, labels], y

In [8]:
# generate points in latent space as input for the generator
# normal noise
def generate_latent_points(latent_dim, n_samples, n_classes=3):
    # generate points in the latent space
#     X_fake = np.random.uniform(0, 1.0, size=[n_samples, latent_dim])
    X_fake = np.random.normal(0,1.0,(n_samples,latent_dim))
    # generate labels
    labels_fake = np.hstack((np.zeros(int(n_samples/3)), np.ones(int(n_samples/3)), 2*np.ones(int(n_samples/3))))
    np.random.shuffle(labels_fake)
    return [reshape(X_fake), keras.utils.to_categorical(labels_fake)]

In [9]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    ecgs = generator.predict([z_input, z_input])
    # create class labels
    y = reshape(np.random.uniform(0, 0.2, n_samples))
#     y = y.reshape(y.shape[0], 1)
#     y = np.zeros((n_samples, 1))
    return [ecgs, labels_input], y

In [10]:
# create and save a plot of generated images
def save_plot(X, n):
    plt.figure(figsize=(10,3))
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(X[i, :, 0])
    plt.show()
    plt.close()

In [11]:
def get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], 1)
    i_S = randint(0, y_S.shape[0], 1)
    i_V = randint(0, y_V.shape[0], 1)
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    return X

In [12]:
def save_new_plot(X_R, z_input, n_batch, name):
    n = 3
    Win = (n_batch//3)
    XX = np.vstack((X_R, z_input[0:n,:,:], z_input[Win:Win+n,:,:], z_input[2*Win:2*Win+n,:,:]))
    plt.figure(figsize=(15,5))
    for i in range(n):
        # subplot(R, C, Plot_No)
        plt.subplot(n+1, n, 1 + i)
        plt.axis('off')
        plt.plot(XX[i,:,:])
    for i in range(n, ((n+1)*(n+1))-(n+1)):
        # define subplot
        plt.subplot(n+1, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(XX[i,:,:])
    # plt.show()
    plt.savefig(name, dpi=75)
    plt.close()

In [13]:
# size of the latent space
latent_dim = 186
# size of the data
data = 186
# classes
classes = 3

n_epochs=10

# multiples of three (three classes) (less thyan 24000)
n_batch=300

# Loss Values
D_L_1 = np.infty
D_L_2 = np.infty
G_L = np.infty

# create the discriminator
d_model = discriminator(data_dim=data)
# d_model = discriminator(data_dim=data, input_classes=classes)

# create the generator
g_model = generator(noise_dim=data, beat_dim=data, out_dim=data)
# g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)

# create the gan
gan_model = create_gan(d_model, g_model)

# folder_name = 'LSTM_RS_True_Tanh/'
folder_name = 'LSTM_RS_True_Tanh_WO_BN/'
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)

plot_model(d_model, to_file=folder_name+'disc.pdf', show_shapes=True)
plot_model(g_model, to_file=folder_name+'gen.pdf', show_shapes=True)
plot_model(gan_model, to_file=folder_name+'gan.pdf', show_shapes=True)


# load image data
X_N, y_N, X_S, y_S, X_V, y_V = load_real_samples()
# # train model
# train(g_model, d_model, gan_model, dataset, latent_dim)

bat_per_epo = int(y_S.shape[0] / n_batch)
half_batch = int(n_batch / 2)
plt.ioff()

filename = folder_name + 'Plots'
if not os.path.isdir(filename):
    os.mkdir(filename)

model_name = folder_name + 'Model/'
if not os.path.isdir(model_name):
    os.mkdir(model_name)

f = open(folder_name + 'Loss.csv', 'w')
f.write('d_loss1, d_loss2, g_loss \n')
f.close()

f = open(folder_name + 'Stats.csv', 'w')
for i in range(3):
    for mtc in metric_to_calculate:
        f.write(str(mtc)+'_'+str(i)+',')
f.write('\n')
f.close()

# manually enumerate epochs
for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
        
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, half_batch)
        # print (X_real.shape, labels_real.shape, y_real.shape)
        # update discriminator model weights
        d_loss1, _ = d_model.train_on_batch([X_real, X_real], y_real)
        
        # generate 'fake' examples
        [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # print (X_fake.shape, labels.shape, y_fake.shape)
        # update discriminator model weights
        d_loss2, _ = d_model.train_on_batch([X_fake, X_fake], y_fake)
        
        # prepare points in latent space as input for the generator
        [z_input, _] = generate_latent_points(latent_dim, n_batch)
        [X, _], _ = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_batch)
        # print (z_input.shape)
        # create inverted labels for the fake samples
        y_gan = reshape(np.random.uniform(0.8, 1, n_batch))
        # update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch([z_input, X], y_gan)
        
        if g_loss < G_L:
            D_L_1 = d_loss1
            D_L_2 = d_loss2
            G_L = g_loss
            g_model.save(model_name + str(i*1000 + j) + '_cgan_generator.h5')

        f = open(folder_name + 'Loss.csv', 'a')
        f.write(str(d_loss1)+','+str(d_loss2)+','+str(g_loss)+'\n')
        f.close()

        if (j+1)%2 == 0:
            print('>%d, %d/%d, d1=%.5f, d2=%.5f g=%.5f' %(i, j, bat_per_epo, d_loss1, d_loss2, g_loss))
            name = filename+'/'+str(i*1000 + j)+'.jpg'
            # generate ECGs
            z_input  = g_model.predict([z_input, X])
            X_R = get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V)
            save_new_plot(X_R, z_input, n_batch, name)
            
        if (j+1)%10 == 0:
            evaluate(X, z_input, classes, metric_to_calculate, n_batch, folder_name, samples=4)

>0, 1/80, d1=0.34833, d2=0.94259 g=0.80824
>0, 3/80, d1=0.38880, d2=0.42068 g=0.92011
>0, 5/80, d1=0.35920, d2=0.35296 g=0.94834
>0, 7/80, d1=0.34121, d2=0.37866 g=1.02340
>0, 9/80, d1=0.32470, d2=0.37652 g=0.88986
>0, 11/80, d1=0.35027, d2=0.43521 g=0.90005
>0, 13/80, d1=0.34306, d2=0.40668 g=0.88467
>0, 15/80, d1=0.33876, d2=0.46067 g=0.93234
>0, 17/80, d1=0.35054, d2=0.36342 g=0.85052
>0, 19/80, d1=0.34525, d2=0.32800 g=0.86523
>0, 21/80, d1=0.34078, d2=0.34106 g=0.80832
>0, 23/80, d1=0.32926, d2=0.33135 g=0.83447
>0, 25/80, d1=0.33720, d2=0.35014 g=0.86140
>0, 27/80, d1=0.35985, d2=0.34843 g=0.87350
>0, 29/80, d1=0.36040, d2=0.33011 g=0.85652
>0, 31/80, d1=0.33209, d2=0.35260 g=0.83378
>0, 33/80, d1=0.33493, d2=0.34566 g=0.81906
>0, 35/80, d1=0.33379, d2=0.35367 g=0.73197
>0, 37/80, d1=0.35473, d2=0.33585 g=0.78836
>0, 39/80, d1=0.34625, d2=0.32604 g=0.73971
>0, 41/80, d1=0.32010, d2=0.32520 g=0.83308
>0, 43/80, d1=0.32754, d2=0.33760 g=0.83749
>0, 45/80, d1=0.33737, d2=0.34624 g=0

>4, 55/80, d1=0.30727, d2=0.33658 g=0.43342
>4, 57/80, d1=0.33412, d2=0.33325 g=0.60157
>4, 59/80, d1=0.31975, d2=0.33349 g=0.60090
>4, 61/80, d1=0.32384, d2=0.33672 g=0.61738
>4, 63/80, d1=0.32349, d2=0.32459 g=0.78633
>4, 65/80, d1=0.32717, d2=0.31616 g=0.82971
>4, 67/80, d1=0.32713, d2=0.32387 g=0.76938
>4, 69/80, d1=0.33036, d2=0.32090 g=0.54139
>4, 71/80, d1=0.32130, d2=0.33567 g=0.81547
>4, 73/80, d1=0.32360, d2=0.30914 g=0.56077
>4, 75/80, d1=0.32683, d2=0.33605 g=0.53162
>4, 77/80, d1=0.32603, d2=0.33628 g=0.60094
>4, 79/80, d1=0.33876, d2=0.31267 g=0.64244
>5, 1/80, d1=0.32979, d2=0.33344 g=0.51007
>5, 3/80, d1=0.30504, d2=0.32615 g=0.32624
>5, 5/80, d1=0.32229, d2=0.32767 g=0.47045
>5, 7/80, d1=0.32021, d2=0.32509 g=0.50715
>5, 9/80, d1=0.33395, d2=0.31735 g=0.58268
>5, 11/80, d1=0.33233, d2=0.34371 g=0.88649
>5, 13/80, d1=0.32090, d2=0.34236 g=0.44625
>5, 15/80, d1=0.34133, d2=0.32781 g=0.74740
>5, 17/80, d1=0.31597, d2=0.32241 g=0.60066
>5, 19/80, d1=0.31315, d2=0.34395 g=0

>9, 29/80, d1=0.31363, d2=0.32482 g=0.35856
>9, 31/80, d1=0.35407, d2=0.31800 g=0.76636
>9, 33/80, d1=0.33855, d2=0.33306 g=0.41492
>9, 35/80, d1=0.34765, d2=0.32508 g=0.61004
>9, 37/80, d1=0.32796, d2=0.33535 g=0.53626
>9, 39/80, d1=0.34840, d2=0.34269 g=0.60302
>9, 41/80, d1=0.32591, d2=0.32805 g=0.75723
>9, 43/80, d1=0.32587, d2=0.33146 g=0.48818
>9, 45/80, d1=0.34293, d2=0.32036 g=0.44948
>9, 47/80, d1=0.30963, d2=0.32929 g=0.52801
>9, 49/80, d1=0.31895, d2=0.33014 g=0.63533
>9, 51/80, d1=0.32004, d2=0.33150 g=0.57442
>9, 53/80, d1=0.33917, d2=0.33113 g=0.82896
>9, 55/80, d1=0.33953, d2=0.31341 g=0.88807
>9, 57/80, d1=0.32905, d2=0.35318 g=0.38999
>9, 59/80, d1=0.36236, d2=0.31763 g=0.98043
>9, 61/80, d1=0.33191, d2=0.33249 g=0.54085
>9, 63/80, d1=0.32733, d2=0.32178 g=0.52972
>9, 65/80, d1=0.30911, d2=0.33512 g=0.44695
>9, 67/80, d1=0.33176, d2=0.34767 g=0.50474
>9, 69/80, d1=0.34171, d2=0.33935 g=0.66417
>9, 71/80, d1=0.32087, d2=0.33090 g=0.47900
>9, 73/80, d1=0.31374, d2=0.3333

In [14]:
# name = filename + '/'+str(i*1000 + j)+'.jpg'
# # generate images
# latent_points, labels = generate_latent_points(latent_dim, n*n)
# # specify labels
# [X, labels], y = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n*n)
# # generate images
# X  = g_model.predict([latent_points, X])
# X_R = get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V)
# X = np.vstack((X_R, X))
# save_new_plot(X, n+1, name)

In [15]:
# f = open(filename+'_Stats.csv', 'a')
# for k,metric in enumerate(label_dict.keys()):    
#     temp_x = test_data[200*(k):200*(k+1),:-1]
#     [z_input, labels_input] = generate_class_specific_latent_input(200, n_classes=n_classes, noise_dim=noise_dim, category=float(metric))
#     z_input = G.predict([z_input, labels_input], verbose=verbose)

#     for j in range(2):
#         plt.plot(z_input[j])
#     plt.savefig(data_dir+str(i)+'_Label_'+str(metric)+'.png')
#     plt.close()
#     plt.clf()

#     results = evaluate(temp_x,z_input,metric_to_calculate)
#     for r in results:
#         f.write(str(r)+',')

# f.write('\n')
# f.close()

In [16]:
# def save_model(model, data_dir, type='G', epoch=1):
#     json_name = data_dir+str(epoch)+'_'+type+'.json'
#     h5name = data_dir+str(epoch)+'_'+type+'.h5'
#     # serialize model to JSON
#     model_json = model.to_json()
#     with open(json_name, "w") as json_file:
#         json_file.write(model_json)
#     # serialize weights to HDF5
#     model.save_weights(h5name)
#     # print("Saved model to disk")
#     del model


In [17]:
# save_model(model=G, data_dir=data_dir, type='G', epoch=i)
# #     # save the generator model
#     g_model.save('cgan_generator.h5')

# callback = [EarlyStopping(monitor='val_AUC', mode='max', verbose=1, patience=Pat),
#          ModelCheckpoint(filepath=str(twelve_lead_model_filename)+'_check_model.h5', 
#                          monitor='val_AUC', verbose=1, save_best_only=True, mode='max'),
#          ReduceLROnPlateau(monitor='val_AUC', factor=0.5, patience=Pat//2, verbose=1, 
#                            mode='max', min_delta=0.0001, cooldown=0, min_lr=0)]

# # model = parallel_NN(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS):
# model = parallel_NN(Window, len(leads), snomed_classes.shape[0])

# model.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
#               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
#               metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),
#                        tf.keras.metrics.Recall(name='Recall'),
#                        tf.keras.metrics.Precision(name='Precision'),
#                        tf.keras.metrics.AUC(num_thresholds=200,summation_method="interpolation",
#                                             name="AUC",dtype=None,curve="ROC",thresholds=None,
#                                             multi_label=True,label_weights=None)])
# history = model.fit(train_generator, steps_per_epoch=train_samples, epochs=EP, verbose=1,
#                 validation_data=val_generator, validation_steps=val_samples, callbacks=callback)


# history_name = output_directory + '/' + twelve_lead_filename
# print (twelve_lead_model_filename, history_name)

# save_model(twelve_lead_model_filename, model)
# write_history(history_name, history.history)